In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

df = pd.read_csv("vegetables_fresh_stale.csv")
df = df[['image_path', 'label']]

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
out = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_cb = ModelCheckpoint(
    "vegetable_quality_checkpoint.h5",
    save_weights_only=False,
    save_best_only=False,
    save_freq='epoch',
    verbose=1
)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint_cb]
)




Found 7562 validated image filenames belonging to 2 classes.
Found 1891 validated image filenames belonging to 2 classes.
Epoch 1/10
  3/237 [..............................] - ETA: 2:24 - loss: 0.9746 - accuracy: 0.5208

c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


237/237 [==============================] - ETA: 0s - loss: 0.3048 - accuracy: 0.8729
Epoch 1: saving model to vegetable_quality_checkpoint.h5


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


237/237 [==============================] - 161s 665ms/step - loss: 0.3048 - accuracy: 0.8729 - val_loss: 0.1844 - val_accuracy: 0.9371
Epoch 2/10
237/237 [==============================] - ETA: 0s - loss: 0.1405 - accuracy: 0.9527
Epoch 2: saving model to vegetable_quality_checkpoint.h5
237/237 [==============================] - 159s 672ms/step - loss: 0.1405 - accuracy: 0.9527 - val_loss: 0.1353 - val_accuracy: 0.9535
Epoch 3/10
237/237 [==============================] - ETA: 0s - loss: 0.1033 - accuracy: 0.9680
Epoch 3: saving model to vegetable_quality_checkpoint.h5
237/237 [==============================] - 159s 670ms/step - loss: 0.1033 - accuracy: 0.9680 - val_loss: 0.1114 - val_accuracy: 0.9630
Epoch 4/10
237/237 [==============================] - ETA: 0s - loss: 0.0813 - accuracy: 0.9746
Epoch 4: saving model to vegetable_quality_checkpoint.h5
237/237 [==============================] - 157s 662ms/step - loss: 0.0813 - accuracy: 0.9746 - val_loss: 0.0981 - val_accuracy: 0.9667
E

In [ ]:
model.save("vegetable_quality_model.keras")


In [3]:
from tensorflow.keras.models import load_model
model = load_model("vegetable_quality_model.keras")
model.save("vegetable_quality_model.h5")


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
